# 출력층 설계 (Output layer)

In [1]:
!pip3 install torch torchvision torchaudio

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.6/204.1 MB 7.6 MB/s eta 0:00:27
    --------------------------------------- 5.0/204.1 MB 13.1 MB/s eta 0:00:16
   - -------------------------------------- 6.8/204.1 MB 11.0 MB/s eta 0:00:18
   - -------------------------------------- 9.7/204.1 MB 11.8 MB/s eta 0:00:17
   -- ------------------------------------- 12.3/204.1 MB 12.1 MB/s eta 0:00:16
   -- ------------------------------------- 15.2/204.1 MB 12.3 MB/s eta 0:00:16
   --- ------------------------------------ 17.3/204.1 MB 11.9 MB/s eta 0:00:16
   --- ------------------------------------ 19.7/204.1 MB 11.9 MB/s eta 0:00:16
   ---- ----------------------------------- 21.5/204.1 MB 11.5 MB/s eta 0:00:16
   ---- ----------------------------------- 22.3/204.1 MB 10.8 MB/s eta 0:00:17
   ---- ----------------------------------- 23.1/204.1 MB 10.3 MB/s eta 0:00:18
   ---- ----------------------------------- 23.6/204.1

### 소프트 맥스 오버플로우 방지

In [3]:
import numpy as np

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def stable_softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)


x = np.array([1000, 1001, 1002])

print(softmax(x))
print(stable_softmax(x))

[nan nan nan]
[0.09003057 0.24472847 0.66524096]


C:\Users\USER\AppData\Local\Temp\ipykernel_432\3860681584.py:4: RuntimeWarning: overflow encountered in exp
  exp_x = np.exp(x)
C:\Users\USER\AppData\Local\Temp\ipykernel_432\3860681584.py:5: RuntimeWarning: invalid value encountered in divide
  return exp_x / np.sum(exp_x)


- PyTorch 라이브러리 함수 사용

In [ ]:
import torch
import torch.nn.functional as F     # nn : neural network의 줄임말. 신경망에 관련된 연산이 모여있는 모듈.

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)   #[1000, 1001, 1002] 정수(int)형 -> 실수형으로 데이터타입을 바꿔주면 됨 

softmax_output = F.softmax(x, dim=0)    # dim: softmax를 적용할 축 
print(softmax_output)

sigmoid_output = torch.sigmoid(x)
print(sigmoid_output)

tensor([0.0900, 0.2447, 0.6652])
tensor([1., 1., 1.])


### 손실 함수와 연계

In [ ]:
import torch
import torch.nn as nn   # 신경망 관련 모듈 
import torch.optim as optim # Optimizer, 최적화관련 모듈 

# 간단한 다중 클래스 분류 모델 정의 (입력값: 5개, 출력: 3클래스)
class SimpleMultiClassModel(nn.Module):
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()   # 부모 클래스 (nn.Module)의 생성자 (__init__)를 호출하여 초기화. super()를 호출하면 부모 클래스의 메서드를 직접 호출할 필요 없이 상속된 메서드를 사용 가능
        self.fc = nn.Linear(5, 3)   #fullyconnected 약자 fc / 선형계산을 수행 

    def forward(self, x):   # 순전파에 관한 함수
        return self.fc(x)   # x를 선형변환 후 반환 
    
model = SimpleMultiClassModel()     # 모델이 메모리에 할당되며, 학습 가능한 가중치들이 초기화
criterion = nn.CrossEntropyLoss()  # 손실 함수 / 모델의 예측값과 실제 정답을 비교하여 손실(loss)을 계산
optimizer = optim.Adam(model.parameters(), lr=0.01)    # 모델의 파라미터를 업데이트. model 파라미터(가중치, 편향)을 최적화 시켜서 업데이트, lr : 학습률

# 데이터 생성 
inputs = torch.randn(4, 5)
labels = torch.tensor([0, 2, 1, 0])

for _ in range(10):
    preds = model(inputs)   # 내부적으로 알아서 forward를 호출(nn.Module을 상속했기 때문에) -> 순전파 
    loss = criterion(preds, labels) # 손실 계산
    print(loss.item())

    optimizer.zero_grad()   # 이전 단계에서 계산된 기울기를 0으로 초기화
    loss.backward()         # 손실에 대한 역전파 수행 (파라미터에 대한 기울기 계산)
    optimizer.step()        # 계산된 기울기를 사용하여 옵티마이저가 모델의 파라미터 업데이트 
# 가중치 업데이트가 반복되며 손실이 줄어드는 것을 볼 수 있다.  이 횟수 (여기서는 10회)를 epoch라고 한다 


1.4190287590026855
1.3863849639892578
1.3545961380004883
1.3236762285232544
1.2936012744903564
1.2643365859985352
1.2358713150024414
1.2082083225250244
1.1813476085662842
1.1552834510803223
